In [1]:
import numpy as np
from matplotlib import pyplot as plt
import skimage
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_dir = "data/table.npz"

In [3]:
def load_data(npz_file):
  bundle = np.load(npz_file, allow_pickle=True)
  num_frames = bundle["num_frames"]

  # Extract poses, intrinsics, images.
  images = []
  camtoworlds = []
  pixtocams = []
  depths = []
  confidences = []
  for i in range(num_frames):
    # Poses.
    w2c = bundle[f"info_{i}"].item()["world_to_camera"]
    camtoworlds.append(np.linalg.inv(w2c))

    # Intrinsics.
    # Swap x and y of principal point values in K.
    K_local = bundle[f"info_{i}"].item()["intrinsics"]
    camera_mat = np.eye(4)
    camera_mat[:3, :3] = K_local.copy()
    camera_mat[0, 2] = K_local[1, 2]
    camera_mat[1, 2] = K_local[0, 2]

    scale_factor = 1920 / 256
    camera_mat = np.diag([1. / scale_factor, 1. / scale_factor, 1., 1.
                  ]).astype(np.float32) @ camera_mat

    pixtocams.append(np.linalg.inv(camera_mat))


    # Images.
    image = (bundle[f"img_{i}"] / 255.).astype(np.float32)
    image = skimage.transform.resize(image, (256, 192))
    images.append(image)

    # Depthmaps.
    depths.append(bundle[f"depth_{i}"])
    confidences.append(bundle[f"conf_{i}"])

  return np.stack(camtoworlds), np.stack(pixtocams), np.stack(images), np.stack(depths), np.stack(confidences)

def world_from_pix_and_z(p2c, c2w, pixel_grid, depth, conf):
  # Homogeneous coords: (u, v, 1, 1/z)
  # Shape: [4, H, W]
  pix_h = np.stack([pixel_grid[0], pixel_grid[1], np.ones_like(depth), 1/depth])
  # Shape [4, H*W]
  pix_h = np.reshape(pix_h, (pix_h.shape[0], np.prod(pix_h.shape[1:])))
  depth = np.reshape(depth, (np.prod(depth.shape)))
  conf = np.reshape(conf, (np.prod(conf.shape)))

  # Convert c2w from OpenGL to OpenCV
  c2w = c2w @ np.diag([1, -1, -1, 1])
  
  points = c2w @ p2c @ pix_h * depth

  return points[:, conf == 2]

def get_pixel_grid(width, height):
  return np.meshgrid(np.arange(width), np.arange(height), indexing='xy')


def save_pointcloud(points, filename):
    # Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
    pcd = o3d.geometry.PointCloud()
    points_formatted = points.T[:, :3]
    pcd.points = o3d.utility.Vector3dVector(points_formatted)
    o3d.io.write_point_cloud(filename, pcd)

In [4]:
camtoworlds, pixtocams, images, depths, confidences = load_data(data_dir)

In [5]:
height, width = images.shape[1:3]
pixel_grid = get_pixel_grid(width, height)
for i in [0, 8]:
  points = world_from_pix_and_z(pixtocams[i], camtoworlds[i], pixel_grid, depths[i], confidences[i])
  save_pointcloud(points, f"test_{i}.ply")


In [8]:
# import imageio
# for i in range(9):
#     filename = f"data/table/images/img_{i:02}.png"
#     imageio.imsave(filename, np.clip(images[i] * 255, 0, 255).astype(np.uint8))